In [2]:
import gym
import collections
import random

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

#Hyperparameters
learning_rate = 0.0005
gamma         = 0.98
buffer_limit  = 50000
batch_size    = 32

class ReplayBuffer():
    def __init__(self):
        self.buffer = collections.deque(maxlen=buffer_limit)
    
    def put(self, transition):
        self.buffer.append(transition)
    
    def sample(self, n):
        mini_batch = random.sample(self.buffer, n)
        s_lst, a_lst, r_lst, s_prime_lst, done_mask_lst = [], [], [], [], []
        
        for transition in mini_batch:
            s, a, r, s_prime, done_mask = transition
            s_lst.append(s)
            a_lst.append([a])
            r_lst.append([r])
            s_prime_lst.append(s_prime)
            done_mask_lst.append([done_mask])

        return torch.tensor(s_lst, dtype=torch.float), torch.tensor(a_lst), \
               torch.tensor(r_lst), torch.tensor(s_prime_lst, dtype=torch.float), \
               torch.tensor(done_mask_lst)
    
    def size(self):
        return len(self.buffer)

class Qnet(nn.Module):
    def __init__(self):
        super(Qnet, self).__init__()
        self.fc1 = nn.Linear(4, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
      
    def sample_action(self, obs, epsilon):
        out = self.forward(obs)
        coin = random.random()
        if coin < epsilon:
            return random.randint(0,1)
        else : 
            return out.argmax().item()
            
def train(q, q_target, memory, optimizer):
    for i in range(10):
        s,a,r,s_prime,done_mask = memory.sample(batch_size)

        q_out = q(s)
        q_a = q_out.gather(1,a)
        max_q_prime = q_target(s_prime).max(1)[0].unsqueeze(1)
        target = r + gamma * max_q_prime * done_mask
        loss = F.smooth_l1_loss(q_a, target)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

def main():
    env = gym.make('CartPole-v1')
    q = Qnet()
    q_target = Qnet()
    q_target.load_state_dict(q.state_dict())
    memory = ReplayBuffer()

    print_interval = 20
    score = 0.0  
    optimizer = optim.Adam(q.parameters(), lr=learning_rate)

    for n_epi in range(10000):
        epsilon = max(0.01, 0.08 - 0.01*(n_epi/200)) #Linear annealing from 8% to 1%
        s = env.reset()
        done = False

        while not done:
            a = q.sample_action(torch.from_numpy(s).float(), epsilon)      
            s_prime, r, done, info = env.step(a)
            done_mask = 0.0 if done else 1.0
            memory.put((s,a,r/100.0,s_prime, done_mask))
            s = s_prime

            score += r
            if done:
                break
            
        if memory.size()>2000:
            train(q, q_target, memory, optimizer)

        if n_epi%print_interval==0 and n_epi!=0:
            q_target.load_state_dict(q.state_dict())
            print("n_episode :{}, score : {:.1f}, n_buffer : {}, eps : {:.1f}%".format(
                                                            n_epi, score/print_interval, memory.size(), epsilon*100))
            score = 0.0
    env.close()

if __name__ == '__main__':
    main()

n_episode :20, score : 10.3, n_buffer : 206, eps : 7.9%
n_episode :40, score : 9.8, n_buffer : 402, eps : 7.8%
n_episode :60, score : 9.3, n_buffer : 588, eps : 7.7%
n_episode :80, score : 9.9, n_buffer : 786, eps : 7.6%
n_episode :100, score : 9.7, n_buffer : 979, eps : 7.5%
n_episode :120, score : 9.7, n_buffer : 1172, eps : 7.4%
n_episode :140, score : 9.8, n_buffer : 1367, eps : 7.3%
n_episode :160, score : 9.6, n_buffer : 1558, eps : 7.2%
n_episode :180, score : 10.0, n_buffer : 1758, eps : 7.1%
n_episode :200, score : 9.5, n_buffer : 1948, eps : 7.0%
n_episode :220, score : 10.4, n_buffer : 2157, eps : 6.9%
n_episode :240, score : 9.9, n_buffer : 2355, eps : 6.8%
n_episode :260, score : 10.2, n_buffer : 2559, eps : 6.7%
n_episode :280, score : 11.4, n_buffer : 2787, eps : 6.6%
n_episode :300, score : 15.7, n_buffer : 3100, eps : 6.5%
n_episode :320, score : 21.6, n_buffer : 3532, eps : 6.4%
n_episode :340, score : 80.8, n_buffer : 5149, eps : 6.3%
n_episode :360, score : 124.0, n

n_episode :2760, score : 168.1, n_buffer : 50000, eps : 1.0%
n_episode :2780, score : 225.2, n_buffer : 50000, eps : 1.0%
n_episode :2800, score : 197.3, n_buffer : 50000, eps : 1.0%
n_episode :2820, score : 208.8, n_buffer : 50000, eps : 1.0%
n_episode :2840, score : 176.8, n_buffer : 50000, eps : 1.0%
n_episode :2860, score : 263.9, n_buffer : 50000, eps : 1.0%
n_episode :2880, score : 229.3, n_buffer : 50000, eps : 1.0%
n_episode :2900, score : 209.5, n_buffer : 50000, eps : 1.0%
n_episode :2920, score : 336.2, n_buffer : 50000, eps : 1.0%
n_episode :2940, score : 281.6, n_buffer : 50000, eps : 1.0%
n_episode :2960, score : 363.1, n_buffer : 50000, eps : 1.0%
n_episode :2980, score : 188.4, n_buffer : 50000, eps : 1.0%
n_episode :3000, score : 121.0, n_buffer : 50000, eps : 1.0%
n_episode :3020, score : 173.2, n_buffer : 50000, eps : 1.0%
n_episode :3040, score : 136.2, n_buffer : 50000, eps : 1.0%
n_episode :3060, score : 199.2, n_buffer : 50000, eps : 1.0%
n_episode :3080, score :

n_episode :5460, score : 267.7, n_buffer : 50000, eps : 1.0%
n_episode :5480, score : 295.9, n_buffer : 50000, eps : 1.0%
n_episode :5500, score : 286.1, n_buffer : 50000, eps : 1.0%
n_episode :5520, score : 314.9, n_buffer : 50000, eps : 1.0%
n_episode :5540, score : 272.1, n_buffer : 50000, eps : 1.0%
n_episode :5560, score : 253.4, n_buffer : 50000, eps : 1.0%
n_episode :5580, score : 190.8, n_buffer : 50000, eps : 1.0%
n_episode :5600, score : 223.0, n_buffer : 50000, eps : 1.0%
n_episode :5620, score : 229.9, n_buffer : 50000, eps : 1.0%
n_episode :5640, score : 233.4, n_buffer : 50000, eps : 1.0%
n_episode :5660, score : 301.6, n_buffer : 50000, eps : 1.0%
n_episode :5680, score : 225.9, n_buffer : 50000, eps : 1.0%
n_episode :5700, score : 244.0, n_buffer : 50000, eps : 1.0%
n_episode :5720, score : 297.2, n_buffer : 50000, eps : 1.0%
n_episode :5740, score : 240.5, n_buffer : 50000, eps : 1.0%
n_episode :5760, score : 187.2, n_buffer : 50000, eps : 1.0%
n_episode :5780, score :

n_episode :8160, score : 107.5, n_buffer : 50000, eps : 1.0%
n_episode :8180, score : 125.3, n_buffer : 50000, eps : 1.0%
n_episode :8200, score : 120.2, n_buffer : 50000, eps : 1.0%
n_episode :8220, score : 116.2, n_buffer : 50000, eps : 1.0%
n_episode :8240, score : 154.7, n_buffer : 50000, eps : 1.0%
n_episode :8260, score : 136.4, n_buffer : 50000, eps : 1.0%
n_episode :8280, score : 184.8, n_buffer : 50000, eps : 1.0%
n_episode :8300, score : 193.3, n_buffer : 50000, eps : 1.0%
n_episode :8320, score : 164.3, n_buffer : 50000, eps : 1.0%
n_episode :8340, score : 200.3, n_buffer : 50000, eps : 1.0%
n_episode :8360, score : 156.3, n_buffer : 50000, eps : 1.0%
n_episode :8380, score : 143.7, n_buffer : 50000, eps : 1.0%
n_episode :8400, score : 258.2, n_buffer : 50000, eps : 1.0%
n_episode :8420, score : 139.5, n_buffer : 50000, eps : 1.0%
n_episode :8440, score : 135.7, n_buffer : 50000, eps : 1.0%
n_episode :8460, score : 131.4, n_buffer : 50000, eps : 1.0%
n_episode :8480, score :